In [19]:
using LinearAlgebra,IntervalArithmetic,ForwardDiff

function orig_is_subset(X,Y)
    for i = 1:size(X)[1]
        if !( issubset(X[i],Y[i]) )
            return false
        end
    end
    return true
end

function orig_intersect(X,Y)
    L = []
    for i = 1:size(X)[1]
        push!(L, intersect(X[i],Y[i]))
        # typeof(intersect(X[i],Y[i])) => Interval{Float64}
    end
    return L
end

function orig_is_empty(X,Y)
    for i = 1:size(X)[1]
        if ( isempty(intersect(X[i],Y[i])) )
            return true
        end
    end
    return false
end

function orig_devide(X)
    #最も長い辺を二等分する
    max_diam=(0..0)
    max_index=0
    X1,X2=[],[]
    for (i,x) in enumerate(X)
        if diam(x)>max_diam
            max_diam=diam(x)
            max_index=i
        end
    end
    
    if diam(X[max_index])<10^(-8)
        return false
    end
    
    X1=X*1 #値をコピーする
    X2=X*1 #値をコピーする
    X1[max_index]= intersect(X[max_index],X[max_index]-radius(X[max_index]))
    X2[max_index]= intersect(X[max_index],X[max_index]+(diam(X[max_index])-radius(X[max_index])))
    return X1,X2
end

function border(K,I)
    c=0.9
    K_width=diam.(K)
    I_width=diam.(I)
    max=0
    for i=1:size(K)[1]
        if (K_width[i]/I_width[i])>max
            max=K_width[i]/I_width[i]
        end
    end
    if max<=0.9
        println("境界条件")
        return true
    else
        return false
    end
end


function allsol(F,X)
    cnt=0
    L = [X] #調査する区間のリスト
    S = [] #見つかった解のリスト
    
    while (size(L)[1] != 0)
        
        X = popfirst!(L) #Lから区間を一つ取り出す
        
        if !( orig_is_subset([(0. ..0.),(0. ..0.)], F(X)) ) 
            print(X)
            println(":非存在条件を満たす")
            continue
        end
        
        c = mid.(X)
        j = ForwardDiff.jacobian(F,X)
        
        if !( orig_is_subset([(0. ..0.),(0. ..0.)], F(c)+j*(X-c)) )
            print(X)
            println(":非存在条件を満たす")
            continue
        end
    
        tmp = ForwardDiff.jacobian(F,c)
        
        R = tmp
        try
            R = inv(tmp)
        catch e
            try
                X1,X2 = orig_devide(X)
                push!(L, X1)
                push!(L, X2)
            catch e
                println(e)
            end
            print(X)
            println(":逆行列の演算に失敗")
            continue
        end
        
        M = Matrix{Float64}(I,size(R))-R*j
        K = c -R*F(c)+M*(X-c)
        
        if (orig_is_empty(K,X))
            print(X)
            println(":解なし")
            continue #解なし
        end
        
        if (orig_is_subset(K,X))　#解あり
            push!(S, K) #解のリストに追加
            print(K)
            println(":解あり")
            continue
        end
        
        # 境界に乗ってた時
        if border(K,X)
            push!(L,K)
            print(K)
            println(":未知")
            continue
        end
        
        try
            print(X)
            println(":未知")
            X1,X2 = orig_devide(orig_intersect(K,X))　#K and X を2分割
            push!(L, X1)
            push!(L, X2)
            continue
        catch e
            println(e)
        end
    end
    @show L
    @show S
    return S
end

X = [(-101. ..100.),(-100. .. 100.)]

f((x1, x2)) = x1^2 + x2^2 - 1
g((x1, x2)) = x1^2 - x2^4

# f((x1,x2)) = x1 + 2*x2 +1
# g((x1,x2)) = x1^2 + 2*(x2^2) -1

# f((x1,x2)) = x1 + 2*x2 +1
# g((x1,x2)) = x1^2 + 2*(x2^2) -1

F((x,y))=[f((x,y));g((x,y))]

li=allsol(F,X)

for i in li
    @show i
end

# 境界に解がのっているとき、区間拡張？
# 重解　＝＞区間幅に制限をつける

Interval{Float64}[[-101, 100], [-100, 100]]:逆行列の演算に失敗
Interval{Float64}[[-101, -0.5], [-100, 100]]:逆行列の演算に失敗
Interval{Float64}[[-0.5, 100], [-100, 100]]:逆行列の演算に失敗
Interval{Float64}[[-101, -0.5], [-100, 0]]:未知
Interval{Float64}[[-101, -0.5], [0, 100]]:未知
Interval{Float64}[[-0.5, 100], [-100, 0]]:未知
Interval{Float64}[[-0.5, 100], [0, 100]]:未知
Interval{Float64}[[-101, -50.75], [-100, 0]]:非存在条件を満たす
Interval{Float64}[[-50.75, -0.5], [-100, 0]]:未知
Interval{Float64}[[-101, -50.75], [0, 100]]:非存在条件を満たす
Interval{Float64}[[-50.75, -0.5], [0, 100]]:未知
Interval{Float64}[[-0.5, 49.75], [-100, 0]]:未知
Interval{Float64}[[49.75, 100], [-100, 0]]:非存在条件を満たす
Interval{Float64}[[-0.5, 49.75], [0, 100]]:未知
Interval{Float64}[[49.75, 100], [0, 100]]:非存在条件を満たす
Interval{Float64}[[-50.75, -0.5], [-100, -50]]:非存在条件を満たす
Interval{Float64}[[-50.75, -0.5], [-50, 0]]:未知
Interval{Float64}[[-50.75, -0.5], [0, 50]]:未知
Interval{Float64}[[-50.75, -0.5], [50, 100]]:非存在条件を満たす
Interval{Float64}[[-0.5, 49.75], [-100, -50]]:非存在条

S = Any[Interval{Float64}[[-0.646194, -0.589688], [-0.806302, -0.766288]], Interval{Float64}[[-0.634253, -0.601137], [-0.798189, -0.77544]], Interval{Float64}[[-0.634253, -0.601137], [0.77544, 0.798189]], Interval{Float64}[[-0.646194, -0.589688], [0.766288, 0.806302]], Interval{Float64}[[0.575481, 0.660537], [-0.816585, -0.756005]], Interval{Float64}[[0.575481, 0.660537], [0.756005, 0.816585]], Interval{Float64}[[0.616261, 0.619764], [-0.787456, -0.784943]], Interval{Float64}[[0.616261, 0.619764], [0.784943, 0.787456]]]
i = Interval{Float64}[[-0.646194, -0.589688], [-0.806302, -0.766288]]
i = Interval{Float64}[[-0.634253, -0.601137], [-0.798189, -0.77544]]
i = Interval{Float64}[[-0.634253, -0.601137], [0.77544, 0.798189]]
i = Interval{Float64}[[-0.646194, -0.589688], [0.766288, 0.806302]]
i = Interval{Float64}[[0.575481, 0.660537], [-0.816585, -0.756005]]
i = Interval{Float64}[[0.575481, 0.660537], [0.756005, 0.816585]]
i = Interval{Float64}[[0.616261, 0.619764], [-0.787456, -0.784943]